In this notebook we will download data from the Amazon Product Reviews Dataset for 4 categories : computers, food & beverages, office supplies and industrial tools. We will then concatenate every category in one dataframe and apply some cleaning functions 

Let's start by importing some libraries !

**Important note:** You should run this notebook for every amazon category file, then concat the results in one dataframe.

In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

Let's download the dataset for Food & Beverage (you need to do the same for every category you wish to include) , you can fin the links here : https://nijianmo.github.io/amazon/index.html

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Grocery_and_Gourmet_Food.json.gz

--2022-01-04 17:13:53--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Grocery_and_Gourmet_Food.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109586529 (105M) [application/octet-stream]
Saving to: ‘meta_Grocery_and_Gourmet_Food.json.gz’

meta_Grocery_and_Go 100%[===================>] 104.51M  96.1MB/s    in 1.1s    

2022-01-04 17:13:54 (96.1 MB/s) - ‘meta_Grocery_and_Gourmet_Food.json.gz’ saved [109586529/109586529]



In [ ]:
### load the meta data

data = []
with gzip.open('meta_Grocery_and_Gourmet_Food.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

287051
{'category': ['Grocery & Gourmet Food', 'Dairy, Cheese & Eggs', 'Cheese', 'Gouda'], 'tech1': '', 'description': ['BEEMSTER GOUDA CHEESE AGED 18/24 MONTHS', 'Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition.'], 'fit': '', 'title': 'Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs', 'also_buy': [], 'tech2': '', 'brand': 'Ariola Imports', 'feature': [], 'rank': '165,181 in Grocery & Gourmet Food (', 'also_view': ['B0000D9MYM', 'B0000D9MYL', 'B00ADHIGBA', 'B00H9OX598', 'B001LM42GY', 'B001LM5TDY'], 'main_cat': 'Grocery', 'similar_item': '', 'date': '', 'price': '$41.91', 'asin': '0681727810', 'imageURL': [], 'imageURLHighRes': []}


In [ ]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

287051


In [ ]:
df[['title','brand']]

,title,brand
0,Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs,Ariola Imports
1,Trim Healthy Mama Xylitol,
2,Letter C - Swarovski Crystal Monogram Wedding ...,Unik Occasions
3,Letter H - Swarovski Crystal Monogram Wedding ...,Other
4,Letter S - Swarovski Crystal Monogram Wedding ...,Unik Occasions
...,...,...
287046,"Caffarel Fruit Jellies, Organic Assorted, 200 ...",Caffarel
287047,"HAMPTON CREEK Spo Just Truffle Mayo, 12 Ounce",Hampton Creek
287048,"HAMPTON CREEK Just Ranch Dressing, 12 Fluid Ounce",Hampton Creek
287049,Gourmet Pure Vanilla Bean Pastes - Organically...,Heilala Vanilla


Next we will write a function that do the following

- Remove any special char
- Keep only titles having the brand in them
- Keep only titles shorter than 13 words
- Convert HTML encodings to characters
- Remove all NAs

In [ ]:
def clean_data(df):
  df = df[['title','brand','asin']]
  special_chars = ['','-','_','#','@','$','=','+','*','£','%',',','.']
  is_branded = df.apply(lambda x : str(x['brand']) in x['title'],axis=1)
  # Remove brands that are numbers; symbols and length of brand is less than 3 words
  is_brand_real = df.apply(lambda x: str(x.brand).isdigit() == False and str(x.brand) not in special_chars,axis=1)
  is_short_brand = df.apply(lambda x: len(str(x.brand).split(" ")) < 3, axis=1)
  # Keep only short titles : shorter than 12 words
  is_title_short = df.apply(lambda x: len(x.title.split(' ')) < 13,axis=1)
  # Remove unformatted rows
  filled_df = df.fillna('')
  is_html = filled_df.title.str.contains('getTime')
  # Replace entity html codes : 
  to_replace = {
      '&quot;': '"',
      '&copy;': '',
      '&reg;' : '',
      '&amp;': '&'
  }
  ## Apply all the filters
  filled_df = filled_df.replace(to_replace)
  df_clean = filled_df[~is_html][is_branded][is_brand_real][is_title_short]
  # Deduplicate 
  df_clean = df_clean.drop_duplicates(subset='title',keep='first')
 
  return df_clean

In [ ]:
cleaned_data = clean_data(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
cleaned_data

,title,brand,asin
10,Lipton Yellow Label Tea (loose tea) - 450g,Lipton,4639725043
12,Wagh Bakri Masala Chai 100's (Pack of 3),Wagh Bakri,4858582000
13,Nutrela High Protein Soya Chunks 200g (Pack of 6),Nutrela,5236363640
15,Organo Gold Cafe Supreme 100% Certified Ganode...,Organo Gold,5463213682
16,MDH Kashmiri Mirch 100g,MDH,5478541265
...,...,...,...
287037,Kashi Dark Cocoa Karma Cereal 16.1oz,Kashi,B01HJ57HNG
287040,Kashi Quinoa Corn &amp; Roasted Pepper Granola...,Kashi,B01HJ5Y05Y
287041,WakeShake Collagen Protein Meal Replacement Sh...,WakeShake,B01HJ7QPKU
287046,"Caffarel Fruit Jellies, Organic Assorted, 200 ...",Caffarel,B01HJFBN9A


Finally, we save the file as an excel file and a csv file.

In [ ]:
name = 'amazon_food'
cleaned_data.to_excel(name+'.xlsx',index=False)
cleaned_data.to_csv(name+'.csv',index=False)